In [1]:
import torch

In [2]:
torch.__version__

'1.4.0'

In [3]:
DEVICE = torch.device('cuda')

In [4]:
rotation = torch.zeros(40, 3, 3, requires_grad=True, device = DEVICE)
translation = torch.zeros(40, 3, requires_grad=True , device = DEVICE)
focal_length = torch.zeros(1, requires_grad=True, device = DEVICE)
distrotion =  torch.zeros(1, requires_grad=True, device = DEVICE)
camera_px = torch.tensor(460.0, requires_grad=False, device = DEVICE)
camera_py = torch.tensor(608.0, requires_grad=False, device = DEVICE)

In [5]:
collect_point2d = []
views_list = []
point3d_list = []
views_count = 0
point3d_count = 0
uv_count = 0
with open('penguin_feature_matching.txt','r') as f:
    views_count, point3d_count, uv_count = f.readline().strip().split(' ')
    collect_point2d = []
    for _ in range(int(uv_count)):
        view, p3d, u,v = list(filter(None,f.readline().strip().split(' ')))
        collect_point2d.append([float(u),float(v)])
        views_list.append(int(view))
        point3d_list.append(int(p3d))

In [6]:
point3d = torch.rand(int(point3d_count), 3, requires_grad=True, device = DEVICE)
point2d = torch.tensor(collect_point2d,requires_grad=False, device = DEVICE)
point3d_index = torch.tensor(point3d_list,dtype=torch.long,requires_grad=False, device = DEVICE)
extrinsic_index = torch.tensor(views_list,dtype=torch.long,requires_grad=False, device = DEVICE)

In [7]:
intrinsic = torch.zeros(3,3, requires_grad=False, device = DEVICE)
intrinsic[0,0] = focal_length
intrinsic[1,1] = focal_length
intrinsic[0,1] = distrotion
intrinsic[0,2] = camera_px
intrinsic[1,2] = camera_py
intrinsic[2,2] = 1.0

In [8]:
extrinsic_pad = torch.zeros(int(uv_count),4,4, requires_grad=False, device = DEVICE)
extrinsic_pad[:,:,3] = 1
extrinsic_pad[:,:3,:3] = rotation[extrinsic_index]
extrinsic_pad[:,3,:3] = translation[extrinsic_index]

In [9]:
point3d_pad = torch.ones(int(uv_count),4,1, requires_grad=False, device = DEVICE)
point3d_pad[:,:3,0] = point3d[point3d_index]

In [10]:
#optimizer = torch.optim.Adam([rotation, translation, focal_length,distrotion,point3d], lr=0.01)
# use per parameter learning rate
optimizer = torch.optim.Adam([
    {'params':focal_length, 'lr':1},
    {'params':rotation},
    {'params':translation},
    {'params':distrotion},
    {'params':point3d},
], lr=0.01) 

In [11]:
for i in range(10000):
    optimizer.zero_grad()
    # Do projection
    extrinsic_pad_point = torch.matmul(extrinsic_pad,point3d_pad)
    extrinsic_pad_point = extrinsic_pad_point[:,:3,:]
    projected_point = torch.matmul(intrinsic,extrinsic_pad_point[:,:3,:])
    projected_point = projected_point[:,:,0]
    uv_map = torch.empty(int(uv_count),2, device = DEVICE)
    uv_map[:,0] = projected_point[:,0] / projected_point[:,2]
    uv_map[:,1] = projected_point[:,1] / projected_point[:,2]
    # compare after projection with point2d
    uv_difference = torch.pow(point2d - uv_map, 2)
    total_loss = uv_difference.sum()
    total_loss.backward(retain_graph=True)
    optimizer.step()
    if i % 100 == 0:
        print("EPOCH %d - loss %f" % (i,total_loss.item()))

EPOCH 0 - loss 1085168768.000000
EPOCH 100 - loss 1085168768.000000
EPOCH 200 - loss 1085168768.000000
EPOCH 300 - loss 1085168768.000000
EPOCH 400 - loss 1085168768.000000
EPOCH 500 - loss 1085168768.000000
EPOCH 600 - loss 1085168768.000000
EPOCH 700 - loss 1085168768.000000
EPOCH 800 - loss 1085168768.000000
EPOCH 900 - loss 1085168768.000000
EPOCH 1000 - loss 1085168768.000000
EPOCH 1100 - loss 1085168768.000000
EPOCH 1200 - loss 1085168768.000000
EPOCH 1300 - loss 1085168768.000000
EPOCH 1400 - loss 1085168768.000000
EPOCH 1500 - loss 1085168768.000000
EPOCH 1600 - loss 1085168768.000000
EPOCH 1700 - loss 1085168768.000000
EPOCH 1800 - loss 1085168768.000000
EPOCH 1900 - loss 1085168768.000000
EPOCH 2000 - loss 1085168768.000000
EPOCH 2100 - loss 1085168768.000000
EPOCH 2200 - loss 1085168768.000000
EPOCH 2300 - loss 1085168768.000000
EPOCH 2400 - loss 1085168768.000000
EPOCH 2500 - loss 1085168768.000000
EPOCH 2600 - loss 1085168768.000000
EPOCH 2700 - loss 1085168768.000000
EPOC